In [1]:
import pandas as pd
import numpy as np
import microdf as mdf
import plotly.express as px
import statsmodels.api as sm
from ubicenter import format_fig

In [15]:
# Read data per person per year.
py = pd.read_csv("data/nys_cps.csv.gz")
py.columns = py.columns.str.lower()
py.inctot.replace({999999999: 0}, inplace=True)
py.adjginc.replace({99999999: 0}, inplace=True)
# 2014 was double-sampled.
py.loc[py.year == 2014, ["asecwt", "spmwt"]] /= 2
# ASEC shows survey year.
py.year -= 1

In [16]:
NYC_COUNTY = pd.DataFrame(columns=["fips", "county", "pop_m"])
NYS_FIPS = 36
# Per https://en.wikipedia.org/wiki/List_of_counties_in_New_York
NYC_COUNTY.loc["Manhattan"] = pd.Series({"fips": 61, "county": "New York", "pop_m": 1.632})
NYC_COUNTY.loc["Brooklyn"] = pd.Series({"fips": 47, "county": "Kings", "pop_m": 2.59})
NYC_COUNTY.loc["The Bronx"] = pd.Series({"fips": 5, "county": "Bronx", "pop_m": 1.435})
NYC_COUNTY.loc["Staten Island"] = pd.Series({"fips": 85, "county": "Richmond", "pop_m": 0.475})
NYC_COUNTY.loc["Queens"] = pd.Series({"fips": 81, "county": "Queens", "pop_m": 2.287})
NYC_COUNTY["full_fips"] = NYS_FIPS * 1000 + NYC_COUNTY.fips

py = py[py.county.isin(NYC_COUNTY.full_fips)][
    ["county", "asecwt", 'age', 'sex', 'race', 'hispan', 'inctot', 'spmwt', 'spmtotres', 'spmthresh',
       'spmfamunit', 'adjginc', 'ftotval', 'spmftotval', 'year', 'spmgeoadj']]
py["adult"] = py.age > 17
py["child"] = py.age <= 17
py["spmratio"] = py.spmtotres / py.spmthresh
py["poor"] = py.spmratio < 1
py["deep_poor"] = py.spmratio < 0.5
# py[]
NYC_COUNTY["asec_pop_m"] = NYC_COUNTY.full_fips.apply(
    lambda x: py[(py.county == x) & (py.year == py.year.max())].asecwt.sum() / 1e6
)
NYC_COUNTY

,fips,county,pop_m,full_fips,asec_pop_m
Manhattan,61,New York,1.632,36061,1.634743
Brooklyn,47,Kings,2.590,36047,2.634859
The Bronx,5,Bronx,1.435,36005,1.586280
Staten Island,85,Richmond,0.475,36085,0.497162
Queens,81,Queens,2.287,36081,1.869408


In [17]:
def year_stats(x):
    return pd.Series({
        "spmgeoadj": mdf.weighted_mean(x, "spmgeoadj", "asecwt"),
        "poverty_rate": mdf.weighted_mean(x, "poor", "asecwt"),
        "deep_poverty_rate": mdf.weighted_mean(x, "deep_poor", "asecwt"),
        "poor_adults": mdf.weighted_sum(x[x.adult], "poor", "asecwt"),
        "deep_poor_adults": mdf.weighted_sum(x[x.adult], "deep_poor", "asecwt"),
        "child_poverty_rate": mdf.weighted_mean(x[x.child], "poor", "asecwt"),
        "child_deep_poverty_rate": mdf.weighted_mean(x[x.child], "deep_poor", "asecwt"),
        "adult_poverty_rate": mdf.weighted_mean(x[x.adult], "poor", "asecwt"),
        "adult_deep_poverty_rate": mdf.weighted_mean(x[x.adult], "deep_poor", "asecwt"),
        "gini": mdf.gini(x, "spmtotres", "asecwt"),
        "population": x.asecwt.sum(),
        "spm_units": x[["spmfamunit", "spmwt"]].drop_duplicates().spmwt.sum(),
        "survey_population": x.shape[0],
        "survey_spm_units": x.spmfamunit.nunique(),
    })
year = py.groupby("year").apply(year_stats)
year


,spmgeoadj,poverty_rate,deep_poverty_rate,poor_adults,deep_poor_adults,child_poverty_rate,child_deep_poverty_rate,adult_poverty_rate,adult_deep_poverty_rate,gini,population,spm_units,survey_population,survey_spm_units
year,,,,,,,,,,,,,,
2009,1.171758,0.238819,0.077847,1406646.140,461700.23,0.285527,0.091285,0.225489,0.074012,0.445619,8018515.38,3462591.14,4011.0,1646.0
2010,1.171034,0.254766,0.083146,1525446.480,511316.47,0.304332,0.091854,0.240661,0.080667,0.420163,8142355.16,3535711.74,3892.0,1653.0
2011,1.177843,0.264718,0.091158,1643151.780,552227.18,0.303672,0.112016,0.253722,0.085271,0.423615,8304166.90,3540372.29,3888.0,1614.0
2012,1.185227,0.283449,0.085348,1724078.310,544069.96,0.346137,0.090276,0.266143,0.083987,0.463452,8266401.85,3654560.71,3741.0,1590.0
2013,1.197308,0.248841,0.080284,1598883.565,526017.99,0.259022,0.077976,0.245993,0.080929,0.455619,8317762.07,3775078.88,3761.0,1624.0
2014,1.205353,0.244156,0.061335,1634360.030,421724.70,0.238525,0.053827,0.245705,0.063401,0.449674,8481922.52,3664382.81,3714.0,1562.0
2015,1.212111,0.209054,0.055558,1332270.600,372160.26,0.251927,0.057029,0.197466,0.055161,0.434709,8570608.64,3675576.36,3327.0,1414.0
2016,1.216385,0.172255,0.048316,1129649.120,337656.81,0.184672,0.040192,0.168928,0.050493,0.446701,8479062.17,3684690.55,3288.0,1406.0
2017,1.214323,0.209325,0.047428,1310488.930,318786.72,0.273164,0.049709,0.192499,0.046827,0.455661,8601995.46,3834281.25,3017.0,1327.0


In [18]:
format_fig(px.line(year.deep_poor_adults))

In [19]:
py.groupby("year").size()

year
2009    4011
2010    3892
2011    3888
2012    3741
2013    3761
2014    3714
2015    3327
2016    3288
2017    3017
2018    3274
2019    2712
dtype: int64

In [24]:
# Use three years of data to smooth out given small sample size.
p = py[py.year.isin([2017, 2018, 2019])].copy()
p[["asecwt", "spmwt"]] /= 3

In [29]:
BUDGET = 1e9  # $1 billion.
REACH = 500e3  # 500,000 adult New Yorkers.

P_S_MERGE_COLS = ["year", "spmfamunit"]
SPMU_COLS = ["spmtotres", "spmthresh", "spmratio", "spmwt", "spmftotval"]
SPMU_AGGS = ["adult", "child"]

s = p.groupby(P_S_MERGE_COLS + SPMU_COLS)[SPMU_AGGS].sum().reset_index()

In [33]:
def phase_out(amount, rate, respect_to):
    return np.maximum(0, amount - respect_to * rate)



def policy(pov_ratio_guarantee, phase_out_rate):
    """ Computes features of a negative income tax.

    Args:
        pov_ratio_guarantee: Maximum benefit as a share of a SPM unit's
            poverty threshold.
        phase_out_rate: Phase-out rate with respect to a SPM unit's resources.
    """
    s["max_transfer"] = s.spmthresh * pov_ratio_guarantee
    s["transfer"] = phase_out(s.max_transfer, phase_out_rate,
                              np.maximum(s.spmtotres, 0))
    p2 = p.merge(s[P_S_MERGE_COLS + ["transfer"]], on=P_S_MERGE_COLS)
    return pd.Series({
        "cost_b": mdf.weighted_sum(s, "transfer", "spmwt") / 1e9,
        "adult_reach": p2[(p2.transfer > 0) & p2.adult].asecwt.sum()
    })
    # s.sort_values("transfer")

In [46]:
policies = mdf.cartesian_product({
    "pov_ratio_guarantee": np.arange(0.01, 0.51, 0.01),
    "phase_out_rate": np.arange(0, 1.01, 0.01)
    })
policies = pd.concat([policies,
    policies.apply(lambda x: policy(x.pov_ratio_guarantee, x.phase_out_rate), axis=1)],
    axis=1)
# Takes ~5min to run.
policies.to_csv("data/policies.csv", index=False)
policies

,pov_ratio_guarantee,phase_out_rate,cost_b,adult_reach
0,0.0,0.00,0.000000,0.000000
1,0.0,0.01,0.000000,0.000000
2,0.0,0.02,0.000000,0.000000
3,0.0,0.03,0.000000,0.000000
4,0.0,0.04,0.000000,0.000000
...,...,...,...,...
5146,0.5,0.96,1.484681,371338.383333
5147,0.5,0.97,1.474007,370007.010000
5148,0.5,0.98,1.463818,356308.060000
5149,0.5,0.99,1.454281,348940.336667


In [86]:
cost_curve = policies[policies.cost_b < 1].sort_values(
    "cost_b", ascending=False
    ).drop_duplicates("pov_ratio_guarantee").sort_values("pov_ratio_guarantee")
cost_curve["constraint"] = "< $1 billion cost"
reach_curve = policies[policies.adult_reach < REACH].sort_values(
    "adult_reach", ascending=False
    ).drop_duplicates("pov_ratio_guarantee").sort_values("pov_ratio_guarantee")
reach_curve["constraint"] = "< 500,000 adult recipients"
cost_reach_curve = pd.concat([cost_curve, reach_curve])
fig = px.line(cost_reach_curve, "pov_ratio_guarantee", "phase_out_rate",
              color="constraint",
              labels={
                  "pov_ratio_guarantee": "Maximum benefit as a share of SPM poverty threshold",
                  "phase_out_rate": "Benefit phase-out rate with respect to SPM resources",
                  "constraint": "Constraint"
                 },
               title="Guaranteed income policies satisfying Yang's Basic Income for NYC constraints")
fig.update_layout(xaxis_tickformat="%", yaxis_tickformat="%")
RATIO = 0.3
PS = 0.5
fig.add_annotation(x=RATIO, y=PS,
                   text="Maximum benefit: 30% of poverty threshold<br>Phase-out rate: 50%")
format_fig(fig)

In [65]:
cost_reach_curve[cost_reach_curve.pov_ratio_guarantee == 0.3]

,pov_ratio_guarantee,phase_out_rate,cost_b,adult_reach,constraint
3080,0.3,0.50,0.997480,478342.19,Cost
3079,0.3,0.49,1.016908,491071.79,Reach


In [59]:

fig = px.line(cost_curve, "pov_ratio_guarantee", "phase_out_rate")
format_fig(fig)

In [81]:
# Determine individualized thresholds.
adult_thresh = s[(s.adult == 1) & (s.child == 0)].spmthresh.max()
child_thresh = ((s.spmthresh - s.adult * adult_thresh) / s.child).max()
print(adult_thresh, child_thresh)
s["spmthresh_ind"] = s.adult * adult_thresh + s.child * child_thresh
s[s.spmthresh == s.spmthresh_ind]

16470.0 8380.0


,spmfamunit,spmtotres,spmthresh,spmratio,spmwt,spmftotval,adult,child,max_transfer,transfer,spmthresh_ind
41,7689001,83471.0,24850.0,3.358994,3104.71,111800,1,1,12425.0,0.0,24850.0
61,7711001,50790.0,16470.0,3.083789,4015.77,73120,1,0,8235.0,0.0,16470.0
238,8035001,24600.0,16470.0,1.493625,2786.97,27653,1,0,8235.0,0.0,16470.0
305,8109001,70425.0,16470.0,4.275956,5872.86,101965,1,0,8235.0,0.0,16470.0
322,8131001,75005.0,16470.0,4.554038,3219.07,110011,1,0,8235.0,0.0,16470.0
328,8142001,59300.0,16470.0,3.600486,4410.93,95210,1,0,8235.0,0.0,16470.0
385,8199001,16700.0,16470.0,1.013965,4949.22,19476,1,0,8235.0,0.0,16470.0
418,8233001,143123.0,16470.0,8.689921,3257.74,229201,1,0,8235.0,0.0,16470.0
449,8301005,21686.0,16470.0,1.316697,2152.04,31002,1,0,8235.0,0.0,16470.0
497,8404001,80899.0,16470.0,4.911900,2254.65,103627,1,0,8235.0,0.0,16470.0


In [87]:
SPMTHRESH_PREDICTORS = ["child", "adult"]
spmthresh_reg = sm.WLS(s.spmthresh, s[SPMTHRESH_PREDICTORS], s.spmwt).fit()
print(spmthresh_reg.params)
s["spmthresh_pred"] = spmthresh_reg.predict(s[SPMTHRESH_PREDICTORS])
s[SPMTHRESH_PREDICTORS + ["spmthresh", "spmthresh_ind", "spmthresh_pred"]]

child     5221.311942
adult    11467.075786
dtype: float64


,child,adult,spmthresh,spmthresh_ind,spmthresh_pred
0,0,3,35030.0,49410.0,34401.227359
1,0,2,22890.0,32940.0,22934.151573
2,0,3,35030.0,49410.0,34401.227359
3,0,1,16240.0,16470.0,11467.075786
4,1,1,24500.0,24850.0,16688.387729
...,...,...,...,...,...
1184,4,2,43460.0,66460.0,43819.399342
1185,2,2,35530.0,49700.0,33376.775457
1186,1,3,32780.0,57790.0,39622.539301
1187,2,2,35530.0,49700.0,33376.775457


In [6]:
# Add cumulative adult population to identify bottom 500,000 adults.
pna = pn[pn.age >= 18].copy()
pna.sort_values("spmftotval", inplace=True)
pna["pop_cum"] = pna.asecwt.cumsum()

spmtotres_thresh = pna[pna.pop_cum < 500000].spmtotres.max()

pna[pna.pop_cum < 500000]

,county,asecwt,age,sex,race,hispan,inctot,spmwt,spmtotres,spmthresh,spmfamunit,adjginc,ftotval,spmftotval,adult,pop_cum
2587,36047,2344.91,76,2,200,0,0,2344.91,0.0,33380.0,8485001,0,0,0,True,2344.91
3459,36061,1986.83,37,1,100,0,0,1986.83,17876.0,30830.0,9057001,0,0,0,True,4331.74
3458,36061,1986.83,37,2,100,0,0,1986.83,17876.0,30830.0,9057001,0,0,0,True,6318.57
1149,36005,5823.38,67,1,200,400,0,5823.38,0.0,16240.0,7668003,0,0,0,True,12141.95
3365,36061,2272.33,62,2,200,0,0,2272.33,-300.0,22890.0,8940001,0,0,0,True,14414.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4122,36081,4959.12,80,1,100,0,12000,4959.12,11858.0,13640.0,9399001,0,12000,12000,True,483357.89
1844,36047,3225.03,55,1,651,0,12000,3225.03,22748.0,39020.0,8059001,12000,12000,12000,True,486582.92
3075,36061,4949.22,72,1,100,0,12000,4949.22,18504.0,16240.0,8726001,0,12000,12000,True,491532.14
1152,36005,3090.58,32,1,200,0,12000,3090.58,30979.0,29080.0,7673001,11152,12000,12000,True,494622.72


In [7]:
sn = pn[["spmfamunit", "spmwt", "spmftotval", "spmtotres", "spmthresh"]].drop_duplicates().sort_values("spmftotval")
sn["spmwt_cum"] = sn.spmwt.cumsum()
spmftotval_thresh = sn[sn.spmwt_cum < 500000].spmftotval.max()
sn[sn.spmftotval <= spmftotval_thresh].spmwt.sum()
sn["ubi"] = np.where(sn.spmftotval <= spmftotval_thresh, 2000, 0)
sn["spmtotres_ubi"] = sn.spmtotres + sn.ubi

In [8]:
mdf.weighted_sum(sn, "ubi", "spmwt")

1004590420.0

In [9]:
pn = pn.merge(sn[["spmfamunit", "spmtotres_ubi"]], on="spmfamunit")


In [10]:
mdf.poverty_gap(sn, "spmtotres_ubi", "spmthresh", "spmwt") / 1e9

5.53524380132

In [11]:
mdf.poverty_gap(sn, "spmtotres", "spmthresh", "spmwt") / 1e9

6.256897865129999

In [12]:
mdf.deep_poverty_rate(pn, "spmtotres", "spmthresh", "asecwt")

0.04859213974110767

In [13]:
mdf.deep_poverty_rate(pn, "spmtotres_ubi", "spmthresh", "asecwt")

0.04356069904096139

In [14]:
mdf.poverty_rate(pn, "spmtotres_ubi", "spmthresh", "asecwt")

0.18354143352276175

In [15]:
mdf.poverty_rate(pn, "spmtotres", "spmthresh", "asecwt")

0.19196885996563795

In [16]:
def pct_chg(base, reform):
    return (reform - base) / base

pct_chg(mdf.poverty_rate(pn, "spmtotres", "spmthresh", "asecwt"),
mdf.poverty_rate(pn, "spmtotres_ubi", "spmthresh", "asecwt"))

-0.04389996609025385

In [17]:
pct_chg(mdf.deep_poverty_rate(pn, "spmtotres", "spmthresh", "asecwt"),
mdf.deep_poverty_rate(pn, "spmtotres_ubi", "spmthresh", "asecwt"))

-0.10354433303314309

In [18]:
pct_chg(mdf.gini(sn, "spmtotres", "spmwt"),
mdf.gini(sn, "spmtotres_ubi", "spmwt"))

-0.01087395327199434

In [19]:
# NYS population: 19.45M official as of 2019.
p[p.statefip == NYS_FIPS].asecwt.sum() / 1e6

19.103431190000002

In [20]:
p[(p.statefip == NYS_FIPS) & p.county.isin(NYC_FIPS)].asecwt.sum() / 1e6

NameError: name 'NYC_FIPS' is not defined

In [22]:
p[p.county.isin([NYS_FIPS * 1000 + i for i in NYC_FIPS])].groupby("county").asecwt.sum()

county
36005    1586280.10
36047    2634859.16
36061    1634742.52
36081    1869407.84
36085     497162.45
Name: asecwt, dtype: float64

In [20]:
p[p.county == 36061].asecwt.sum()

1634742.52